## Use this Notebook to work with *model3* Served with TF Serving Container

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
test_images = test_images / 255.0
test_images_res = test_images.reshape((test_images.shape[0], 28, 28, 1))

In [ ]:
class_names = ['T-shirt', 'Trousers', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
tf_serving_host = 'tf_model3'
model_name = 'my_model3'
# 'serving_predict_url' as in https://www.tensorflow.org/tfx/tutorials/serving/rest_simple
model_predict_url = 'http://'+tf_serving_host+':8501/v1/models/'+model_name+':predict'

In [ ]:
!pip install requests

In [ ]:
import requests
import json

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks(np.arange(0,10,1))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')
    
def predict_remote(tf_serving_model_predict_url, images, index):
    # get a single image from image list and turn it into one element list
    img = (np.expand_dims(images[index],0))
    # build a request payload, 'instances' key contains actual Image to classify 
    request_json = json.dumps({"signature_name": "serving_default", "instances": img.tolist()})
    request_headers = {"content-type": "application/json"}
    response_json = requests.post(tf_serving_model_predict_url, data=request_json, headers=request_headers)
    prediction = json.loads(response_json.text)['predictions']
    plt.figure(figsize=(10,3))
    plt.subplot(1,2,1)
    plot_image(0, prediction, (np.expand_dims(test_labels[index],0)), (np.expand_dims(test_images[index],0)))
    plt.subplot(1,2,2)
    plot_value_array(0, prediction,  (np.expand_dims(test_labels[index],0)))
    plt.xticks(range(10), class_names, rotation=45)
    plt.show()
    return np.argmax(prediction[0])

In [ ]:
i = 0
class_names[predict_remote(model_predict_url, test_images_res, i)]

In [ ]:
i = 17
class_names[predict_remote(model_predict_url, test_images_res, i)]

In [ ]:
i = 12
class_names[predict_remote(model_predict_url, test_images_res, i)]

In [ ]:
i = 8239
class_names[predict_remote(model_predict_url, test_images_res, i)]